In [1]:
import os
import torch
from random import randint
import numpy as np
import gym
from gym.spaces import Box, Discrete, MultiBinary
import cv2
import math
import matplotlib.pyplot as plt
import time
import pandas as pd
from tqdm.notebook import tqdm
from multiprocessing import Pool
import time
# import scipy
# import scipy.stats
from my_function import save_variable, load_variable, cor_to_ind, ind_to_cor, cor_to_nor, nor_to_cor, lat_to_cor, a_star_search
from my_ppo import PPO, Memory, ActorCritic, Critic, Actor
from my_env import MyEnv
device = torch.device('cpu')
if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")

Device set to : NVIDIA GeForce RTX 3080


In [ ]:
num_ppo = 80#
num_env = 200
test_time = 4 #
suc_flags = np.zeros((num_ppo, num_env))
ave_speed = np.zeros((num_ppo, num_env))
dur_time = np.zeros((num_ppo, num_env))
i = -1
T1 = time.perf_counter()
for k in range(0,79):
    path = 'ppoF/ppo' + str((k+1)*1000)
    ppo = load_variable(path)
    i += 1
    env = MyEnv(train=False)
    for env_ind in range(num_env):
        for chance_i  in range(test_time):
            if suc_flags[i][env_ind] == 1:
                continue
            state_1, state_2 = env.reset(rand = env_ind)
            temp_chance_speeds = []
            memory = Memory()
            for t in range(800):
                action = ppo.select_action(state_1, state_2, memory)
                state_1, state_2, reward, done= env.step(action)
                speed = np.linalg.norm(state_1[2:4]*5, ord = 2)
                temp_chance_speeds.append(speed)
                if done:
                    suc_flags[i][env_ind] = 1
                    dur_time[i][env_ind] = t+1
                    ave_speed[i][env_ind] = np.mean(temp_chance_speeds)
                    break
            if t == 799 and chance_i == test_time -1:
                suc_flags[i][env_ind] = 0
                dur_time[i][env_ind] = t+1
                ave_speed[i][env_ind] = np.mean(temp_chance_speeds)
    T2 = time.perf_counter()
    print('{} Time: {:.2f}'.format(path, T2-T1))
    T1 = time.perf_counter()

save_variable(suc_flags, 'suc_flags')
save_variable(ave_speed , 'ave_speed')
save_variable(dur_time, 'dur_time')
# os.system("/usr/bin/shutdown")

/root/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


ppoF/ppo1000 Time: 1366.02
ppoF/ppo2000 Time: 1175.19
ppoF/ppo3000 Time: 1163.16


In [ ]:
# np.mean(dur_time[0])

In [ ]:
# sum(suc_flags[0])/200